# Model Setup

In [19]:
import torchtext
from torchtext.datasets import IMDB
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
from attack import SubstitutionAttack
from torch.utils.data import DataLoader



In [20]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from imdb import IMDBDataset

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
from tqdm import tqdm

In [22]:
# Code from https://tanmay17061.medium.com/load-pre-trained-glove-embeddings-in-torch-nn-embedding-layer-in-under-2-minutes-f5af8f57416a
vocab,embeddings = [],[]
with open('./glove/glove.6B.50d.txt','rt') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    embeddings.append(i_embeddings)
vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:10])

pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

#insert embeddings for pad and unk tokens at top of embs_npa.
embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
print(embs_npa.shape)


['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a']
(400002, 50)


In [ ]:
with open('vocab_npa.npy','rb') as f:
    vocab_npa = np.load(f)

with open('embs_npa.npy','rb') as f:
    emps_npa = np.load(f)

In [23]:
# Simple model

class GloveModel(nn.Module):
    def __init__(self, embs_npa, embed_dim=50, hidden_dim=100, classes=2, threshold=0.5):
        super(GloveModel, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())
        self.embedding_layer.weight.requires_grad = True
        self.threshold = 0.5
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), 
            nn.ReLU(), 
            nn.Linear(hidden_dim, classes),
            nn.Softmax(dim = 2))

    def forward(self, x):
        embedding = self.embedding_layer(x)
        logits = self.classifier(embedding)
        return torch.mean(logits, dim=1, keepdim=True).squeeze()
        

In [24]:
tokenizer = get_tokenizer('basic_english')
train_dataset = IMDBDataset("aclImdb/train", vocab_npa, tokenizer)
test_dataset = IMDBDataset("aclImdb/test", vocab_npa, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=100)
test_loader = DataLoader(test_dataset, batch_size=100)

# Training

In [25]:
#schedulers
device = torch.device('cuda:2' if torch.cuda.is_available() else "cpu")
def epsilon_scheduler(epsilon):

    epsilon_schedule = []
    step = epsilon / 25000

    for i in range(5000):
        epsilon_schedule.append(i * step)
    
    for i in range(20001):
        epsilon_schedule.append(epsilon)
    
    return epsilon_schedule

def kappa_scheduler():

    schedule = 2001 * [1]
    kappa_value = 1.0
    step = 0.5/23000

    for i in range(23000):
        kappa_value -= step
        schedule.append(kappa_value)
    return schedule


In [52]:
def bound_propagation(model: GloveModel, init_bounds):
    l, u = init_bounds
    l = l.to(device).float()
    u = u.to(device).float()
    bounds = [init_bounds]
    
    for layer in model.classifier:

        if isinstance(layer, nn.Linear):
            new_l = 0.5 * (l + u) @ (layer.weight).T  +  layer.bias - 0.5 * (u - l) @ torch.abs(layer.weight).T # pos weights * l + neg_weights * u minimizes lower bound
            new_u = 0.5 * (l + u) @ (layer.weight).T  +  layer.bias + 0.5 * (u - l) @ torch.abs(layer.weight).T # pos weights * l + neg_weights * u minimizes lower bound
        if isinstance(layer, nn.ReLU):
            new_l = l.clamp(min = 0)
            new_u = u.clamp(min = 0)

            unstable = (l < 0) & (u > 0)
            new_l[unstable] = 0  
        if isinstance(layer, nn.Softmax):
            exp_lower = torch.exp(l)
            exp_upper = torch.exp(u)

            mask = torch.ones((2, 2), device=l.device)
            mask.fill_diagonal_(0)
            
            sum_others_upper = torch.matmul(exp_upper, mask)  # (64, 512, 2)
            
            sum_others_lower = torch.matmul(exp_lower, mask)  # (64, 512, 2)
            
            new_l = exp_lower / (exp_lower + sum_others_upper)
            new_u = exp_upper / (exp_upper + sum_others_lower)

            new_l = new_l.mean(dim=1)
            new_u = new_u.mean(dim=1)
            
        l = new_l
        u = new_u
        bounds.append([new_l, new_u])
    return bounds

def robust_train_loop(train_loader, model, epsilon_scheduler, kappa_scheduler, batch_counter, optimizer, attack_obj):
    robust_err = 0
    total_combined_loss = 0

    for indices, label in tqdm(train_loader):
        indices,label = indices.to(device), label.to(device)
        # Fit loss 
        y_prediction = model(indices)
        fit_loss = nn.CrossEntropyLoss()(y_prediction, label)

        # Spec Loss
        # initial_bound = torch.zeros((2, indices.shape[0], indices.shape[1], 50)).to(device)
        initial_bound = attack_obj.get_bounds(indices, epsilon_scheduler[batch_counter])
        bounds = bound_propagation(model, initial_bound)

        lower, upper = bounds[-1]
        cert_loss = torch.max(nn.CrossEntropyLoss()(lower, label),
                               nn.CrossEntropyLoss()(upper, label))
        
        robust_preds = lower[label] < upper[label ^ 1]
        robust_err += robust_preds.sum()
        #combined loss
        combined_loss = (1 - kappa_scheduler[batch_counter])*(fit_loss) + (kappa_scheduler[batch_counter])*(cert_loss)
        total_combined_loss += combined_loss.item()

        batch_counter += 1
    
        if optimizer:
            optimizer.zero_grad()
            combined_loss.backward()
            optimizer.step()

    return robust_err / len(train_loader.dataset) , total_combined_loss/len(train_loader.dataset)



def test_model(model, test_loader):
    correct = 0
    for j, (text, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        _, preds = torch.argmax(logits, 0)
        correct += (preds == labels).sum().item()
    return correct/len(test_loader.dataset)

def test_robust_model(model, test_loader, epsilon, attack_obj):
    robust_err = 0
    for indices, label in train_loader:
        indices,label = indices.to(device), label.to(device)
        initial_bound = torch.zeros((2, indices.shape[0], indices.shape[1], 50))
        for i, index in enumerate(indices):
            bound = attack_obj.get_bounds(index, epsilon)
            initial_bound[:, i] = bound
        bounds = bound_propagation(model, initial_bound)

        lower, upper = bounds[-1]
        robust_preds = lower[label] < upper[label ^ 1]
        robust_err += robust_preds.sum()

    return robust_err/test_loader.dataset

In [47]:
attack_obj = SubstitutionAttack(vocab_npa, embs_npa, precomputed_bounds = "bounds_dict.json")

In [40]:
import json
d = attack_obj.bounds_dict
print(type(d))
with open("bounds_dict.json", "wb") as f:
    torch.save(d, f)

<class 'dict'>


In [53]:
glove = GloveModel(embs_npa)
glove = glove.to(device)
torch.manual_seed(42)
opt = optim.Adam(glove.parameters(), lr=1e-3)

EPSILON = 0.1
EPSILON_TRAIN = 0.2
epsilon_schedule = epsilon_scheduler(EPSILON_TRAIN)
kappa_schedule = kappa_scheduler()
batch_counter = 0

print("Epoch   ", "Combined Loss", "Test Acc", "Test Robust Err", sep="\t")
losses = []
test_errs = []
robust_errs = []
training_robust_errs = []
start = time.time()

for t in tqdm(range(20)):
    print(t)
    training_robust_err, combined_loss = robust_train_loop(train_loader, glove, epsilon_schedule, kappa_schedule, batch_counter, opt, attack_obj)
    batch_counter += 250
    
    if t == 24:  #decrease learning rate after 25 epochs
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-4

    if t == 49:  #decrease learning rate after 49 epochs
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-5
    
    # test_err = test_model(glove, test_loader)
    # robust_err = test_robust_model(glove, test_loader, EPSILON, attack_obj)
    # test_errs.append(test_err)
    # robust_errs.append(robust_err)
    # print(*("{:.6f}".format(i) for i in (t, combined_loss, test_err, robust_err)), sep="\t")
    # training_robust_errs.append(training_robust_err)
    # losses.append(combined_loss)

end_time = time.time() - start
print(f'Time: {end_time}')


Epoch   	Combined Loss	Test Acc	Test Robust Err


  0%|          | 0/20 [00:00<?, ?it/s]

0


  0%|          | 0/20 [58:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# losses
# test_errs
# robust_errs
# training_robust_errs
import matplotlib.pyplot as plt
plt.plot(range(len(losses)), losses, label = "loss")
plt.plot(range(len(test_errs)), test_errs, label = "test_err")
plt.plot(range(len(robust_errs)), robust_errs, label = "robust_err")
plt.xlabel("Epochs")
# plt.plot(range(len(training_robust_errs)), training_robust_errs, label=)
plt.legend()
plt.show()
## Verification
def box_verification(model, test_loader, epsilons):
    total_images = len(test_loader.dataset)
    verified_robust = torch.zeros(len(epsilons))
    adversarial_examples = {}

    for idx, (indices, label) in enumerate(test_loader):
        indices,label = indices.to(device), label.to(device)
        robust_err = 0
        for epsilon in epsilons:
            initial_bound = torch.zeros((2, indices.shape[0], indices.shape[1], 50))
            for i, index in enumerate(indices):
                bound = attack_obj.get_bounds(index, epsilon)
                initial_bound[:, i] = bound
            bounds = bound_propagation(model, initial_bound)

            lower, upper = bounds[-1]
            robust_preds = lower[label] < upper[label ^ 1]
            robust_err += robust_preds.sum()
        verified_robust[idx] = robust_err / len(test_loader.dataset)

    return verified_robust

epsilons = np.linspace(0.01, 0.1, 10)
verified_accuracy, adversarial_examples = box_verification(glove, test_loader, epsilons)

for e, a in zip(epsilons, verified_accuracy):
    print(f'Accuracy for {e}: {a}')



In [51]:
glove.classifier[0].weight.dtype

torch.float32